In [1]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.infogram import H2OInfogram
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators import H2ORandomForestEstimator
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule
pd.options.mode.chained_assignment = None

(CVXPY) Jul 11 10:32:32 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 11 10:32:32 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 11 10:32:32 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="binary_classification", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='SC', #option: KVB, SC
    imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=False,
    balance_class=True,
    random_state=2022)

feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
Missing values in train before imputation: 128
Missing values in train after imputation: 0
Missing values in test before imputation: 24
Missing values in test after imputation: 0
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Train.csv
Label Encoder, 0: Non-responders (No Response), 1: Responders(Good, Moderate)
save file to: /Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/.csv_temp/Test.csv
1.0    204
0.0    184
Name: DrugResponse_binary, dtype: int64


In [4]:
train_set, train_loc = dataset.get_train()
test_set, test_loc = dataset.get_test()

In [5]:
train_set

,grp,init_group,age,gender,final_education,race_grp,ethnicity,weight,BMI,height,...,md_global_assess,pt_global_assess,di,pt_pain,usresultsCRP,usresultsIgA,usresultsIgG,usresultsIgM,DAS28_CRP_0M,DrugResponse_binary
0,1,0,61.0,0,0,7,1,240.0,39.933728,65.0,...,55.0,50.0,0.250000,60.0,1.00,44.0,948.0,51.0,3.719481,0.0
1,1,0,50.0,0,0,7,1,250.0,39.151259,67.0,...,35.0,27.0,0.500000,20.0,1.33,402.0,1290.0,89.0,3.809990,0.0
2,2,0,69.0,1,4,7,1,127.0,17.710970,71.0,...,50.0,40.0,0.571429,70.0,12.89,551.0,672.0,51.0,4.465034,0.0
3,1,0,61.0,0,2,7,1,145.0,26.517950,62.0,...,40.0,5.0,0.857143,60.0,1.31,164.0,902.0,72.0,3.439129,0.0
4,2,0,75.0,0,4,7,1,164.0,27.288047,65.0,...,50.0,50.0,0.857143,80.0,14.15,280.0,1520.0,211.0,4.963210,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383,2,0,66.0,0,2,7,1,240.0,39.933728,65.0,...,55.0,60.0,0.375000,45.0,1.84,164.0,907.0,98.0,4.754878,1.0
384,2,0,72.0,0,0,7,1,172.0,29.520508,64.0,...,65.0,20.0,1.714286,15.0,23.63,182.0,846.0,102.0,5.179392,1.0
385,2,0,55.0,1,0,7,1,195.0,27.976531,70.0,...,40.0,50.0,0.000000,65.0,4.45,195.0,1340.0,40.0,4.202136,1.0
386,1,0,68.0,0,4,0,1,182.0,34.384843,61.0,...,85.0,85.0,0.375000,85.0,1.15,262.0,1110.0,106.0,6.365518,1.0


In [6]:
# Start the H2O cluster (locally)
h2o.init()

# Import a sample binary outcome train/test set into H2O
train_h2o = h2o.upload_file(str(train_loc))
test_h2o = h2o.upload_file(str(test_loc))

# Identify predictors and response
predictors = train_h2o.columns[:-1]
# y = "DAS28_CRP_3M"
response = "DrugResponse_binary"

for feature in dataset.categorical:
    train_h2o[feature] = train_h2o[feature].asfactor()
    test_h2o[feature] = test_h2o[feature].asfactor()
train_h2o[response] = train_h2o[response].asfactor()
test_h2o[response] = test_h2o[response].asfactor()

train, valid = train_h2o.split_frame(ratios=[.8], seed=1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "13.0.2" 2020-01-14; Java(TM) SE Runtime Environment (build 13.0.2+8); Java HotSpot(TM) 64-Bit Server VM (build 13.0.2+8, mixed mode, sharing)
  Starting server from /Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmph3uyrgly
  JVM stdout: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmph3uyrgly/h2o_gaskell_started_from_python.out
  JVM stderr: /var/folders/c9/_bb79xy945g7bvvn1rl6zsfr0000gn/T/tmph3uyrgly/h2o_gaskell_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,08 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 15 days
H2O_cluster_name:,H2O_from_python_gaskell_cuecn0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
# Build and train the model:
RAdrugs_gbm = H2OGradientBoostingEstimator(
    model_id = "GBM_grid_1_AutoML_1_20220711_223756_model_8"
    nfolds = 10,
    ntrees = 50,
    max_depth = 20,
    min_rows=10,
#     balance_classes = True,
#     max_after_balance_size = 1.0,
    calibrate_model=True,
    calibration_frame=valid,
    histogram_type="Random",
    binomial_double_trees=True)

In [8]:
RAdrugs_drf.train(x=predictors,
                  y=response,
                  training_frame=train,
                  validation_frame=valid)

drf Model Build progress: |

/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:208: RuntimeWarning: Dropping bad and constant columns: [pres_imuran, pres_minocin, init_group, num_tnf, num_nontnf]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1657593186997_1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,100.0,20211.0,3.0,8.0,5.72,5.0,15.0,11.45




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.17019641378668804
RMSE: 0.412548680505329
LogLoss: 0.5170005825809474
Mean Per-Class Error: 0.44557237378772774
AUC: 0.6706244901658661
AUCPR: 0.8587013997672299
Gini: 0.34124898033173223

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6052792346811662: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-07-11 22:33:34,5.969 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-07-11 22:33:34,6.072 sec,1.0,0.441386,1.533991,0.604691,0.814387,1.083360,0.232323,0.487107,3.419807,0.452110,0.724254,0.878788,0.241379
2,,2022-07-11 22:33:35,6.102 sec,2.0,0.460651,1.405420,0.540784,0.771425,1.052406,0.251534,0.456446,1.176089,0.497565,0.761038,0.878788,0.241379
3,,2022-07-11 22:33:35,6.127 sec,3.0,0.451456,1.428468,0.585843,0.787053,1.062526,0.255208,0.462777,1.215481,0.403409,0.721011,0.878788,0.241379
4,,2022-07-11 22:33:35,6.147 sec,4.0,0.445721,1.202640,0.585784,0.804886,1.120618,0.246377,0.453437,0.673935,0.439123,0.728569,1.318182,0.224138
5,,2022-07-11 22:33:35,6.163 sec,5.0,0.450253,1.470495,0.563526,0.785639,1.060894,0.250000,0.457691,0.695700,0.413149,0.721441,1.318182,0.241379
6,,2022-07-11 22:33:35,6.176 sec,6.0,0.445426,1.285696,0.548410,0.790484,1.103329,0.240175,0.457918,0.701206,0.418019,0.721703,1.318182,0.241379
7,,2022-07-11 22:33:35,6.187 sec,7.0,0.437920,0.840789,0.559990,0.811552,1.183957,0.236287,0.451973,0.648244,0.448864,0.733348,1.318182,0.241379
8,,2022-07-11 22:33:35,6.195 sec,8.0,0.426767,0.679382,0.603633,0.836088,1.242424,0.233333,0.450021,0.646624,0.453734,0.731367,1.318182,0.241379
9,,2022-07-11 22:33:35,6.205 sec,9.0,0.429470,0.687695,0.591705,0.826508,1.227807,0.238683,0.445901,0.621907,0.466721,0.735118,1.318182,0.241379



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,BMI,95.221985,1.000000,0.121474
1,usresultsIgM,77.312767,0.811921,0.098628
2,DAS28_CRP_0M,70.784920,0.743367,0.090300
3,weight,62.865238,0.660197,0.080197
4,usresultsCRP,48.935802,0.513913,0.062427
5,seatedbp1,38.695244,0.406369,0.049363
6,pt_global_assess,36.450279,0.382793,0.046499
7,grp,35.139751,0.369030,0.044828
8,usresultsRF,29.880039,0.313793,0.038118
9,usresultsCCP3,26.278404,0.275970,0.033523



See the whole table with table.as_data_frame()


In [14]:
# Eval performance:
perf = RAdrugs_drf.model_performance()

In [11]:
# Generate predictions on a validation set (if necessary):
pred = RAdrugs_drf.predict(valid)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [35]:
pred

predict,p0,p1,cal_p0,cal_p1
1,0.238866,0.761134,0.265009,0.734991
1,0.309549,0.690451,0.283913,0.716087
1,0.019237,0.980763,0.211624,0.788376
1,0.033695,0.966305,0.214882,0.785118
1,0.0772251,0.922775,0.224913,0.775087
1,0.018126,0.981874,0.211375,0.788625
1,0.0472126,0.952787,0.217962,0.782038
1,0.0356065,0.964393,0.215316,0.784684
1,0,1,0.207344,0.792656
1,0.018126,0.981874,0.211375,0.788625


In [39]:
pred.head(100)

predict,p0,p1,cal_p0,cal_p1
1,0.238866,0.761134,0.265009,0.734991
1,0.309549,0.690451,0.283913,0.716087
1,0.019237,0.980763,0.211624,0.788376
1,0.033695,0.966305,0.214882,0.785118
1,0.0772251,0.922775,0.224913,0.775087
1,0.018126,0.981874,0.211375,0.788625
1,0.0472126,0.952787,0.217962,0.782038
1,0.0356065,0.964393,0.215316,0.784684
1,0,1,0.207344,0.792656
1,0.018126,0.981874,0.211375,0.788625


In [43]:
valid[response].head(100)

DrugResponse_binary
1
1
0
0
0
1
1
1
1
1
